In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
import pickle

# Data Clean Up

In [24]:
dups = pd.read_csv('pdb_data_no_dups.csv')

In [25]:
seq = pd.read_csv('pdb_data_seq.csv')

In [26]:
prot = dups.merge(seq, how='inner', on='structureId')

In [27]:
prot = prot[prot['macromoleculeType_x']=='Protein']

In [28]:
prot = prot[prot['classification']==prot['classification']]

In [29]:
prot = prot[prot['sequence']==prot['sequence']]

In [46]:
counts = prot.classification.value_counts()
print(counts[:10])

HYDROLASE                        46336
TRANSFERASE                      36424
OXIDOREDUCTASE                   34321
IMMUNE SYSTEM                    15615
LYASE                            11682
HYDROLASE/HYDROLASE INHIBITOR    11218
TRANSCRIPTION                     8919
VIRAL PROTEIN                     8495
TRANSPORT PROTEIN                 8371
VIRUS                             6972
Name: classification, dtype: int64


In [47]:
types = np.asarray(counts[(counts > 6971)].index)

In [50]:
prot = prot[prot.classification.isin(types)]

In [49]:
prot.to_csv('protein_data.csv')

In [51]:
prot[:180000].to_csv('protein_train.csv')
prot[180001:].to_csv('protein_test.csv')

# With scikit-learn (start here!)

In [18]:
data = pd.read_csv('protein_data.csv')

In [19]:
X_train, X_test,y_train,y_test = train_test_split(data['sequence'], data['classification'], test_size = 0.04, random_state = 1)

In [20]:
vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (4,4))

In [21]:
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

In [201]:
prediction = dict()

# Naive Bayes Model

model = MultinomialNB()
model.fit(X_train_df, y_train)
NB_pred = model.predict(X_test_df)

print(accuracy_score(NB_pred, y_test))

0.8573324485733245


In [202]:
print(classification_report(y_test, NB_pred, target_names = types))

                               precision    recall  f1-score   support

                    HYDROLASE       0.81      0.82      0.82      1839
                  TRANSFERASE       0.65      0.81      0.72       447
               OXIDOREDUCTASE       0.94      0.83      0.88       623
                IMMUNE SYSTEM       0.93      0.89      0.91       465
                        LYASE       0.95      0.90      0.93      1362
HYDROLASE/HYDROLASE INHIBITOR       0.88      0.79      0.83       351
                TRANSCRIPTION       0.84      0.93      0.89      1476
                VIRAL PROTEIN       0.90      0.83      0.86       328
            TRANSPORT PROTEIN       0.86      0.69      0.77       345
                        VIRUS       0.89      0.88      0.88       299

                     accuracy                           0.86      7535
                    macro avg       0.87      0.84      0.85      7535
                 weighted avg       0.86      0.86      0.86      7535



# Without scikit-learn

In [4]:
train = pd.read_csv('protein_train.csv')

In [5]:
test = pd.read_csv('protein_test.csv')

In [61]:
class NaiveBayes:
    
    def __init__(self, use_mle):
        self.label_counts = {}
        self.feature_counts = {}
        self.use_mle = use_mle
        
    def get_features_from_sequence(self, seq, n):
        feats = set()
        for i in range(len(seq)-(n-1)):
            feats.add(seq[i: i+n])
        return feats
        
    def fit(self, seqs, classifs, n):
        for i in range(len(seqs)):
            self.label_counts[classifs[i]] = self.label_counts.get(classifs[i], 0) + 1
            feats = self.get_features_from_sequence(seqs[i], n)
            for feat in feats:
                tup = (classifs[i], feat)
                self.feature_counts[tup] = self.feature_counts.get(tup, 0) + 1
    
    def predict(self, seqs, total, n):
        final_preds = []
        for seq in seqs:
            preds = {}
            features = self.get_features_from_sequence(seq, n)
            for classif in self.label_counts.keys():
                preds[classif] = 0
                num = 0
                den = 0
                if not self.use_mle:
                    num = 1
                    den = len(self.label_counts)
                classif_count = self.label_counts.get(classif, 0)
                for feat in features:
                    preds[classif] += np.log((self.feature_counts.get((classif, feat), 0)+num)/(classif_count+den))
                preds[classif] += np.log(classif_count/total)
            classif = max(preds, key=preds.get)
            final_preds.append(classif)
        return final_preds
    
    def print_stats(self, predicts, classifs, precision):
        correct = {}
        classif_dict = {}
        values = classifs
        if precision:
            values = predicts
        for i in range(len(predicts)):
            classif_dict[values[i]] = classif_dict.get(values[i], 0) + 1
            if predicts[i] == classifs[i]:
                correct[values[i]] = correct.get(values[i], 0) + 1
        avg = 0
        for classif in classif_dict.keys():
            p = correct[classif]/classif_dict[classif]
            print(classif, p)
            avg += p
        print('Average: ', avg/len(classif_dict.keys()))
    
    def print_precisions(self, predicts, classifs):
        self.print_stats(predicts, classifs, True)
            
    def print_recalls(self, predicts, classifs):
        self.print_stats(predicts, classifs, False)
        
    def print_accuracy(self, predicts, classifs):
        count = 0
        for i in range(len(predicts)):
            if predicts[i] == classifs[i]:
                count += 1
        print(count/len(test['classification']))

In [62]:
nb = NaiveBayes(False)

In [63]:
nb.fit(train['sequence'], train['classification'], 4)

In [64]:
predicts = nb.predict(test['sequence'], len(test['classification']), 4)

In [65]:
nb.print_precisions(predicts, test['classification'])

TRANSCRIPTION 0.851063829787234
HYDROLASE 0.4473358116480793
TRANSFERASE 0.5768463073852296
OXIDOREDUCTASE 0.620866590649943
HYDROLASE/HYDROLASE INHIBITOR 0.41935483870967744
IMMUNE SYSTEM 0.8993212669683258
LYASE 0.9482758620689655
TRANSPORT PROTEIN 0.8746268656716418
VIRAL PROTEIN 0.738831615120275
VIRUS 0.6050420168067226
Average:  0.6981565004816094


In [66]:
nb.print_recalls(predicts, test['classification'])

TRANSCRIPTION 0.36613272311212813
HYDROLASE/HYDROLASE INHIBITOR 0.21487603305785125
TRANSFERASE 0.7363057324840764
HYDROLASE 0.6880559085133418
OXIDOREDUCTASE 0.8649722001588562
VIRAL PROTEIN 0.42322834645669294
IMMUNE SYSTEM 0.726691042047532
TRANSPORT PROTEIN 0.5069204152249135
LYASE 0.5513784461152882
VIRUS 0.10419681620839363
Average:  0.5182757663379074


In [67]:
nb.print_accuracy(predicts, test['classification'])

0.6148227969348659
